In [3]:
from flask import Flask
from flask import jsonify

from flask_cors import CORS


app = Flask(__name__)
CORS(app)

In [4]:
# Load EDA
import json
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [5]:
#Store the data
df=pd.read_csv('udemy.csv')

In [6]:
#show the data
df.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9


In [7]:
#create a function to combine the important columns/features
def create_book_id_column(data):
    features=[]
    for i in range (0,data.shape[0]):
        features.append( i )
    
    return features

In [8]:
#Create a column to store the combined features
df['book_id']=create_book_id_column(df)
#show the data
df.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,book_id
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18,0
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9,1


In [9]:
#create a function to combine the important columns/features
def combine_feature(data):
    features=[]
    for i in range (0,data.shape[0]):
        features.append( data['course_title'][i])
    
    return features

In [10]:
#Create a column to store the combined features
df['combined_features']=combine_feature(df)
#show the data
df.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,book_id,combined_features
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18,0,Ultimate Investment Banking Course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9,1,Complete GST Course & Certification - Grow You...


In [11]:
#convert the text from the new column to a matrix of word counts
cm= CountVectorizer().fit_transform(df['combined_features'])
print(cm)

  (0, 3328)	1
  (0, 1771)	1
  (0, 369)	1
  (0, 806)	1
  (1, 806)	1
  (1, 725)	1
  (1, 1523)	1
  (1, 608)	1
  (1, 1515)	1
  (1, 3572)	1
  (1, 545)	1
  (1, 2476)	1
  (2, 1312)	1
  (2, 2135)	1
  (2, 1362)	1
  (2, 532)	1
  (2, 224)	1
  (2, 229)	1
  (2, 766)	1
  (3, 1312)	1
  (3, 411)	1
  (3, 3219)	1
  (3, 2517)	1
  (3, 222)	1
  (3, 1685)	1
  :	:
  (3679, 3482)	1
  (3679, 300)	1
  (3679, 938)	1
  (3679, 3532)	1
  (3680, 229)	1
  (3680, 1895)	1
  (3680, 524)	1
  (3680, 3366)	1
  (3680, 2450)	1
  (3681, 3572)	1
  (3681, 823)	1
  (3681, 2274)	1
  (3681, 3482)	1
  (3681, 214)	1
  (3681, 1095)	1
  (3681, 853)	1
  (3681, 238)	1
  (3682, 411)	1
  (3682, 3219)	1
  (3682, 1528)	1
  (3682, 524)	1
  (3682, 3366)	1
  (3682, 3483)	1
  (3682, 680)	1
  (3682, 2143)	1


In [12]:
#Get the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#print the scores
#print(cs)

In [13]:
def search_term_not_found(term,df):
    result_df = df[df['course_title'].str.contains(term,case=False)]
    return result_df

In [14]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

In [15]:
def recommend_courses(title,cosine_similarity_matrix,df,num_of_rec=10):
    result_df = search_term_not_found(title,df)
    for row in result_df.iterrows():
        title = row[1][1]
        print(title)
        break;
                
    # indices of the course
    course_indices = pd.Series(df.index,index=df['course_title'])
    index=course_indices[title]
    print(index)        
    #create a list of tuples in the form (book_id,similarity scores)
    scores = list(enumerate(cs[index]))
    #print(scores)
    #sort the list of similar books in descending order
    try:
        sorted_scores = sorted(scores,key=lambda x:x[1],reverse= True)
        #show the sorted scores
        #sorted_scores
        #create a loop to print the first 5 books from the sorted list
        j=0
        # print('The 5 most recommended books to '+title+' are:\n')
        result=[]
        for item in sorted_scores:
            data= {
                        "course_title" : df[df.book_id==item[0]]['course_title'].values[0],
                        "url" :df[df.book_id==item[0]]['url'].values[0],
                        "price" :df[df.book_id==item[0]]['price'].values[0],
                        "num_subscribers" : df[df.book_id==item[0]]['num_subscribers'].values[0],
                        "num_reviews" : df[df.book_id==item[0]]['num_reviews'].values[0],
                        "num_lectures" : df[df.book_id==item[0]]['num_lectures'].values[0],
                        "duration" : df[df.book_id==item[0]]['content_duration'].values[0]
                    }
            print(data)
            result.append(data)
            j=j+1
            if j>30:
                break;
    except:
        result=[]
    return  json.dumps(result, default=np_encoder)

In [16]:
recommend_courses('python',cs,df,5)

Python Algo Trading: Sentiment Trading with News
14
{'course_title': 'Python Algo Trading: Sentiment Trading with News', 'url': 'https://www.udemy.com/hedge-fund-strategy-trading-with-sentiment-analysis/', 'price': 200, 'num_subscribers': 294, 'num_reviews': 19, 'num_lectures': 42, 'duration': '7 hours'}
{'course_title': 'Python Algo Trading: FX Trading with Oanda', 'url': 'https://www.udemy.com/python-algo-trading-fx-trading-with-oanda/', 'price': 200, 'num_subscribers': 453, 'num_reviews': 42, 'num_lectures': 33, 'duration': '3 hours'}
{'course_title': 'Python Algo Stock Trading: Automate Your Trading!', 'url': 'https://www.udemy.com/algorithmic-stock-trading-bootcamp-automate-your-trading/', 'price': 95, 'num_subscribers': 1165, 'num_reviews': 21, 'num_lectures': 41, 'duration': '2.5 hours'}
{'course_title': 'Quantitative Trading Analysis with Python', 'url': 'https://www.udemy.com/quantitative-trading-analysis-with-python/', 'price': 50, 'num_subscribers': 256, 'num_reviews': 17, '

'[{"course_title": "Python Algo Trading: Sentiment Trading with News", "url": "https://www.udemy.com/hedge-fund-strategy-trading-with-sentiment-analysis/", "price": 200, "num_subscribers": 294, "num_reviews": 19, "num_lectures": 42, "duration": "7 hours"}, {"course_title": "Python Algo Trading: FX Trading with Oanda", "url": "https://www.udemy.com/python-algo-trading-fx-trading-with-oanda/", "price": 200, "num_subscribers": 453, "num_reviews": 42, "num_lectures": 33, "duration": "3 hours"}, {"course_title": "Python Algo Stock Trading: Automate Your Trading!", "url": "https://www.udemy.com/algorithmic-stock-trading-bootcamp-automate-your-trading/", "price": 95, "num_subscribers": 1165, "num_reviews": 21, "num_lectures": 41, "duration": "2.5 hours"}, {"course_title": "Quantitative Trading Analysis with Python", "url": "https://www.udemy.com/quantitative-trading-analysis-with-python/", "price": 50, "num_subscribers": 256, "num_reviews": 17, "num_lectures": 49, "duration": "5.5 hours"}, {"

In [17]:
@app.route('/get/rcmd/<string:title>')
def index(title):
    return recommend_courses(title,cs,df,5)
     

@app.route('/')
def home():
    return ' call this --> /get/rcmd/<course> '

if __name__=="__main__" :
    app.run(port='4545')

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4545/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jul/2021 04:44:47] "GET /get/rcmd/python HTTP/1.1" 200 -


Python Algo Trading: Sentiment Trading with News
14
{'course_title': 'Python Algo Trading: Sentiment Trading with News', 'url': 'https://www.udemy.com/hedge-fund-strategy-trading-with-sentiment-analysis/', 'price': 200, 'num_subscribers': 294, 'num_reviews': 19, 'num_lectures': 42, 'duration': '7 hours'}
{'course_title': 'Python Algo Trading: FX Trading with Oanda', 'url': 'https://www.udemy.com/python-algo-trading-fx-trading-with-oanda/', 'price': 200, 'num_subscribers': 453, 'num_reviews': 42, 'num_lectures': 33, 'duration': '3 hours'}
{'course_title': 'Python Algo Stock Trading: Automate Your Trading!', 'url': 'https://www.udemy.com/algorithmic-stock-trading-bootcamp-automate-your-trading/', 'price': 95, 'num_subscribers': 1165, 'num_reviews': 21, 'num_lectures': 41, 'duration': '2.5 hours'}
{'course_title': 'Quantitative Trading Analysis with Python', 'url': 'https://www.udemy.com/quantitative-trading-analysis-with-python/', 'price': 50, 'num_subscribers': 256, 'num_reviews': 17, '

127.0.0.1 - - [14/Jul/2021 04:45:05] "GET /get/rcmd/java HTTP/1.1" 200 -


Learning Dynamic Website Design - PHP MySQL and JavaScript
2484
{'course_title': 'Learning Dynamic Website Design - PHP MySQL and JavaScript', 'url': 'https://www.udemy.com/learning-dynamic-website-design/', 'price': 50, 'num_subscribers': 47886, 'num_reviews': 285, 'num_lectures': 125, 'duration': '12.5 hours'}
{'course_title': 'Projects in PHP and MySQL', 'url': 'https://www.udemy.com/the-complete-web-development-course-learn-by-building-apps/', 'price': 60, 'num_subscribers': 10606, 'num_reviews': 570, 'num_lectures': 88, 'duration': '23 hours'}
{'course_title': 'Practical Understanding of PHP and MySQL', 'url': 'https://www.udemy.com/practical-understanding-of-php-and-myqsl-demo/', 'price': 0, 'num_subscribers': 12201, 'num_reviews': 217, 'num_lectures': 24, 'duration': '5.5 hours'}
{'course_title': 'PHP MySQL: Learn PHP MySQL with Project', 'url': 'https://www.udemy.com/php-mysql-project-learning/', 'price': 20, 'num_subscribers': 2314, 'num_reviews': 77, 'num_lectures': 25, 'dura

...

{'course_title': 'Complete Website & CMS in PHP & MySQL From Scratch!', 'url': 'https://www.udemy.com/cms-admin-panel-in-php-mysql/', 'price': 20, 'num_subscribers': 3357, 'num_reviews': 82, 'num_lectures': 48, 'duration': '12.5 hours'}
{'course_title': 'Learn E-Commerce Website in PHP & MySQL From Scratch!', 'url': 'https://www.udemy.com/ecommerce-website-in-php-mysqli/', 'price': 0, 'num_subscribers': 29990, 'num_reviews': 1342, 'num_lectures': 87, 'duration': '17.5 hours'}
{'course_title': 'JavaScript Complete Guide to learning JavaScript', 'url': 'https://www.udemy.com/javascript-complete-guide-to-learning-javascript/', 'price': 200, 'num_subscribers': 1296, 'num_reviews': 22, 'num_lectures': 72, 'duration': '5.5 hours'}
Ultimate Investment Banking Course
0
{'course_title': 'Ultimate Investment Banking Course', 'url': 'https://www.udemy.com/ultimate-investment-banking-course/', 'price': 200, 'num_subscribers': 2147, 'num_reviews': 23, 'num_lectures': 51, 'duration': '1.5 hours'}
{'

127.0.0.1 - - [14/Jul/2021 07:16:20] "GET /get/rcmd/investment HTTP/1.1" 200 -



{'course_title': 'Financial Accounting - The Ultimate Beginner Course', 'url': 'https://www.udemy.com/financial-accounting-the-ultimate-beginner-course/', 'price': 100, 'num_subscribers': 9, 'num_reviews': 2, 'num_lectures': 101, 'duration': '10 hours'}
{'course_title': 'Managerial Accounting - The Ultimate Beginner Course!', 'url': 'https://www.udemy.com/managerial-accounting-the-ultimate-beginner-course/', 'price': 100, 'num_subscribers': 11, 'num_reviews': 0, 'num_lectures': 121, 'duration': '11 hours'}
{'course_title': 'The Ultimate Vue JS 2 Developers Course', 'url': 'https://www.udemy.com/vuejs-2-essentials/', 'price': 195, 'num_subscribers': 1703, 'num_reviews': 292, 'num_lectures': 178, 'duration': '13 hours'}
{'course_title': 'PHP with PDO - ULTIMATE Crash Course', 'url': 'https://www.udemy.com/php-with-pdo-ultimate-crash-course/', 'price': 50, 'num_subscribers': 1248, 'num_reviews': 171, 'num_lectures': 18, 'duration': '1.5 hours'}
Learning Dynamic Website Design - PHP MySQL

127.0.0.1 - - [14/Jul/2021 07:16:29] "GET /get/rcmd/php HTTP/1.1" 200 -



{'course_title': 'Complete Website & CMS in PHP & MySQL From Scratch!', 'url': 'https://www.udemy.com/cms-admin-panel-in-php-mysql/', 'price': 20, 'num_subscribers': 3357, 'num_reviews': 82, 'num_lectures': 48, 'duration': '12.5 hours'}
{'course_title': 'Learn E-Commerce Website in PHP & MySQL From Scratch!', 'url': 'https://www.udemy.com/ecommerce-website-in-php-mysqli/', 'price': 0, 'num_subscribers': 29990, 'num_reviews': 1342, 'num_lectures': 87, 'duration': '17.5 hours'}
{'course_title': 'JavaScript Complete Guide to learning JavaScript', 'url': 'https://www.udemy.com/javascript-complete-guide-to-learning-javascript/', 'price': 200, 'num_subscribers': 1296, 'num_reviews': 22, 'num_lectures': 72, 'duration': '5.5 hours'}
